<a href="https://colab.research.google.com/github/gjwnsdud5076/Jun/blob/master/crop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!mkdir test

In [4]:
import os
os.chdir('/content/test')
os.getcwd()

'/content/test'

test용 jpg 업로드하기

In [5]:
from google.colab import files
uploaded = files.upload()

Saving jiheum (10).jpg to jiheum (10).jpg
Saving jiheum (11).jpg to jiheum (11).jpg
Saving jiheum (12).jpg to jiheum (12).jpg


trained weight best 업로드하기

In [10]:
os.chdir('/content')
os.getcwd()

'/content'

In [11]:
from google.colab import files
uploaded = files.upload()

Saving best.pt to best.pt


In [12]:
!git clone  'https://github.com/ultralytics/yolov5.git'

Cloning into 'yolov5'...
remote: Enumerating objects: 3271, done.
remote: Total 3271 (delta 0), reused 0 (delta 0), pack-reused 3271
Receiving objects: 100% (3271/3271), 6.48 MiB | 33.02 MiB/s, done.
Resolving deltas: 100% (2183/2183), done.


In [13]:
!pip install -qr '/content/yolov5/requirements.txt'  # install dependencies

     |████████████████████████████████| 276kB 12.5MB/s 


detect.py 바꾸기

In [45]:
os.chdir('/content/test/yolov5')
os.getcwd()

'/content/test/yolov5'

In [46]:
uploaded = files.upload()

In [48]:
os.chdir('/content')
os.getcwd()

'/content'

In [15]:
output = !python yolov5/detect.py --source "/content/test/*.jpg" --weights '/content/best.pt' --save-txt
print(output)

["Namespace(agnostic_nms=False, augment=False, classes=None, conf_thres=0.25, device='', exist_ok=False, img_size=640, iou_thres=0.45, name='exp', project='runs/detect', save_conf=False, save_txt=True, source='/content/test/*.jpg', update=False, view_img=False, weights=['/content/best.pt'])", 'Using torch 1.7.0+cu101 CUDA:0 (Tesla T4, 15079MB)', '', 'Fusing layers... ', 'Model Summary: 232 layers, 7246518 parameters, 0 gradients, 16.8 GFLOPS', 'image 1/3 /content/test/jiheum (10).jpg: 352x640 3 texts, Done. (0.011s)', 'image 2/3 /content/test/jiheum (11).jpg: 352x640 3 texts, Done. (0.015s)', 'image 3/3 /content/test/jiheum (12).jpg: 352x640 7 texts, Done. (0.010s)', 'Results saved to runs/detect/exp2', '3 labels saved to runs/detect/exp2/labels', 'Done. (0.099s)']


crop

In [22]:
path='/content/runs/detect/exp2/labels/jiheum (10).txt'
name=path.split("/")[6].split(".")
name=name[0]
print(name)


jiheum (10)


In [19]:
import re
path='/content/runs/detect/exp2/labels/jiheum (10).txt'
myfile=open(path,'r')
lines=myfile.readlines()
pattern= "0"

boxes=[]
for line in lines:
  if re.search(pattern,line):
    Cord_Raw=line
    Cord=Cord_Raw.split( )
    boxes.append(Cord)

print(boxes)

[['0', '494', '169', '525', '205'], ['0', '197', '164', '242', '212'], ['0', '531', '169', '562', '206']]


In [44]:
import cv2
img = cv2.imread("/content/runs/detect/exp2/jiheum (10).jpg")
cnt = 0
for box in boxes:
    a=box[2] 
    b=box[4]
    c=box[1]
    d=box[3]
    crop_img = img[int(a):int(b), int(c):int(d)]
    cv2.imwrite(name+"_"+str(cnt)+".jpg",crop_img)
    cnt += 1


169 205 494 525
164 212 197 242
169 206 531 562


In [50]:
import torch
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from LeNet5_model import LeNet5_model, Config
#import matplotlib.pyplot as pp


if __name__ == "__main__":
    cfg = Config()

    # GPU 사용이 가능하면 사용하고, 불가능하면 CPU 활용
    #print("GPU Available:", torch.cuda.is_available())
    device = torch.device("cpu")  #"cuda" if torch.cuda.is_available() else
    print("device:", device)
    # GPU 사용시
    # if torch.cuda.is_available():
    # torch.cuda.device(0)

    # 모델 생성
    model = LeNet5_model()


    if torch.cuda.is_available():
        model = model.to(device)

    model.eval()

    save_path = "/content/epoch_21.pth"
    checkpoint = torch.load(save_path)
    model.load_state_dict(checkpoint['model_state_dict'])


    classes = ('O', 'P', 'Q', 'R', 'S', 'T', 'U')

    transforms_test = transforms.Compose([
        transforms.Resize([42, 42]),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ])

    imgs = ImageFolder('./crop_img', transform=transforms_test)
    print("img shape",imgs[0][0].shape)
    print("imgs:", imgs)
    test_loader = DataLoader(imgs, batch_size=1)


    print("test_loader:", test_loader)
    print(test_loader.dataset)

    for thisimg, label in test_loader:
        pred = model.forward(thisimg.to(device))
        _, top_pred = torch.topk(pred, k=1, dim=-1)
        top_pred = top_pred.squeeze(dim=1)
        print("--------------------------------------")
        print("truth:", classes[label])
        print("model prediction:", classes[top_pred])


device: cpu
img shape torch.Size([3, 42, 42])
imgs: Dataset ImageFolder
    Number of datapoints: 1
    Root location: ./crop_img
    StandardTransform
Transform: Compose(
               Resize(size=[42, 42], interpolation=PIL.Image.BILINEAR)
               ToTensor()
               Normalize(mean=(0.4914, 0.4822, 0.4465), std=(0.2023, 0.1994, 0.201))
           )
test_loader: <torch.utils.data.dataloader.DataLoader object at 0x7f468db79eb8>
Dataset ImageFolder
    Number of datapoints: 1
    Root location: ./crop_img
    StandardTransform
Transform: Compose(
               Resize(size=[42, 42], interpolation=PIL.Image.BILINEAR)
               ToTensor()
               Normalize(mean=(0.4914, 0.4822, 0.4465), std=(0.2023, 0.1994, 0.201))
           )
--------------------------------------
truth: P
model prediction: P
